In [1]:
import random
import math
import pandas as pd
import numpy as np
import json
from keras.models import model_from_json
from numpy import array

Using TensorFlow backend.


In [2]:
import pathlib

# define the path
directory = pathlib.Path('Interpolation')

# define the pattern
currentCompany = 'CHKP'

for currentFile in directory.glob(currentCompany):
    print(currentFile)


fileNum = 'InterpolatedNumWeek' + str(currentCompany) + '.csv'
modelNum = 'modelNum' + str(currentCompany) + '.json'
weightsNum = 'modelNum' + str(currentCompany) + '.h5'
#r2ScoreNum = str(currentCompany) + '_R2ScoreNum' +  '.txt'

fileDen = 'InterpolatedDenWeek' + str(currentCompany) + '.csv'
modelDen = 'modelDen' + str(currentCompany) + '.json'
weightsDen = 'modelDen' + str(currentCompany) + '.h5'
#r2ScoreDen = str(currentCompany) + '_R2ScoreDen' +  '.txt'

Interpolation\CHKP


In [3]:
class boundaries:
    max_D = {}
    min_D = {}
    max_N = {}
    min_N = {}

    def __init__(self, max_D, min_D, max_N, min_N): 
        self.max_D = max_D 
        self.min_D = min_D
        self.max_N = max_N
        self.min_N = min_N

def read_csv():
    #Leyendo archivo
    df_D = pd.read_csv(currentFile/fileDen)
    df_N = pd.read_csv(currentFile/fileNum)
    global ufcf, dfcf, max_D, max_N, min_D, min_N
    global x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12
    #Denominador
    max_D = {'D Revenue':df_D['D Revenue'].max(), 'U CR':df_D['U CR'].max(), 'D OE':df_D['D OE'].max(), 
       'U NOI':df_D['U NOI'].max(),'U CAPEX':df_D['U CAPEX'].max(), 'U WK':df_D['U WK'].max()} 
    min_D = {'D Revenue':df_D['D Revenue'].min(), 'U CR':df_D['U CR'].min(), 'D OE':df_D['D OE'].min(), 
       'U NOI':df_D['U NOI'].min(),'U CAPEX':df_D['U CAPEX'].min(), 'U WK':df_D['U WK'].min()}
    #Numerador
    max_N = {'U Revenue':df_N['U Revenue'].max(), 'D CR':df_N['D CR'].max(), 'U OE':df_N['U OE'].max(), 
       'D NOI':df_N['D NOI'].max(),'D CAPEX':df_N['D CAPEX'].max(), 'D WK':df_N['D WK'].max()} 
    min_N = {'U Revenue':df_N['U Revenue'].min(), 'D CR':df_N['D CR'].min(), 'U OE':df_N['U OE'].min(), 
       'D NOI':df_N['D NOI'].min(),'D CAPEX':df_N['D CAPEX'].min(), 'D WK':df_N['D WK'].min()}
   
    filas_d, columnas_d = df_D.count()-1, len(df_D.columns)-1
    dataset_D = df_D.values
    #Variables a pasar a la funcion generate_population
    dfcf = float(dataset_D[filas_d, columnas_d][1])
    filas_n, columnas_n = df_N.count()-1, len(df_N.columns)-1
    dataset_N = df_N.values
    ufcf = float(dataset_N[filas_d, columnas_d][1])
    
    boundaries_x = boundaries(max_D, min_D, max_N, min_N)
        
    
    ##Calculo de las x
    #Numerador
    x1 = df_N['U Revenue'].corr(df_N['U FCF'])
    x2 = df_N['D CR'].corr(df_N['U FCF'])
    x3 = df_N['U OE'].corr(df_N['U FCF'])
    x4 = df_N['D NOI'].corr(df_N['U FCF'])
    x5 = df_N['D CAPEX'].corr(df_N['U FCF'])
    x6 = df_N['D WK'].corr(df_N['U FCF'])
    #Denominador
    x7 = df_D['D Revenue'].corr(df_D['D FCF'])
    x8 = df_D['U CR'].corr(df_D['D FCF'])
    x9 = df_D['D OE'].corr(df_D['D FCF'])
    x10 = df_D['U NOI'].corr(df_D['D FCF'])
    x11 = df_D['U CAPEX'].corr(df_D['D FCF'])
    x12 = df_D['U WK'].corr(df_D['D FCF'])

    return boundaries_x


def generate_population(boundarie, size):
    population = []
    min_N = boundarie.min_N
    max_N = boundarie.max_N
    min_D = boundarie.min_D
    max_D = boundarie.max_D
    for i in range(size):
        individual = {
            #Numerador
            "w1": random.uniform(min_N['U Revenue'], max_N['U Revenue']),
            "w2": random.uniform(min_N['D CR'], max_N['D CR']),
            "w3": random.uniform(min_N['U OE'], max_N['U OE']),
            "w4": random.uniform(min_N['D NOI'], max_N['D NOI']),
            "w5": random.uniform(min_N['D CAPEX'], max_N['D CAPEX']),
            "w6": random.uniform(min_N['D WK'], max_N['D WK']),
            #Denominador
            "w7": random.uniform(min_D['D Revenue'], max_D['D Revenue']),
            "w8": random.uniform(min_D['U CR'], max_D['U CR']),
            "w9": random.uniform(min_D['D OE'], max_D['D OE']),
            "w10": random.uniform(min_D['U NOI'], max_D['U NOI']),
            "w11": random.uniform(min_D['U CAPEX'], max_D['U CAPEX']),
            "w12": random.uniform(min_D['U WK'], max_D['U WK']),
        }    
        population.append(individual)
    return population

def apply_function(individual):
    # Numerador
    # load json and create model
    json_file = open(currentFile/modelNum, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model_num = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model_num.load_weights(currentFile/weightsNum)
    # Denominador
    json_file = open(currentFile/modelDen, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model_den = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model_den.load_weights(currentFile/weightsDen)    
    w1 = individual["w1"]
    w2 = individual["w2"]
    w3 = individual["w3"]
    w4 = individual["w4"]
    w5 = individual["w5"]
    w6 = individual["w6"]
    w7 = individual["w7"]
    w8 = individual["w8"]
    w9 = individual["w9"]
    w10 = individual["w10"]
    w11 = individual["w11"]
    w12 = individual["w12"]
    #Numerador
    w1_std = (w1-min_N['U Revenue'])/(max_N['U Revenue']-min_N['U Revenue'])
    w2_std = (w2-min_N['D CR'])/(max_N['D CR']-min_N['D CR'])
    w3_std = (w3-min_N['U OE'])/(max_N['U OE']-min_N['U OE'])
    w4_std = (w4-min_N['D NOI'])/(max_N['D NOI']-min_N['D NOI'])
    w5_std = (w5-min_N['D CAPEX'])/(max_N['D CAPEX']-min_N['D CAPEX'])
    w6_std = (w6-min_N['D WK'])/(max_N['D WK']-min_N['D WK'])
    #Denominador
    w7_std = (w7-min_D['D Revenue'])/(max_D['D Revenue']-min_D['D Revenue'])
    w8_std = (w8-min_D['U CR'])/(max_D['U CR']-min_D['U CR'])
    w9_std = (w9-min_D['D OE'])/(max_D['D OE']-min_D['D OE'])
    w10_std = (w10-min_D['U NOI'])/(max_D['U NOI']-min_D['U NOI'])
    w11_std = (w11-min_D['U CAPEX'])/(max_D['U CAPEX']-min_D['U CAPEX'])
    w12_std = (w12-min_D['U WK'])/(max_D['U WK']-min_D['U WK'])
    w_numerador = [[w1_std, w2_std, w3_std, w4_std, w5_std, w6_std]]
    w_denominador = [[w7_std, w8_std, w9_std, w10_std, w11_std, w12_std]]
    Xnewnum = array(w_numerador)
    Xnewden = array(w_denominador)
    # make a prediction
    numerador = loaded_model_num.predict(Xnewnum) 
    denominador = loaded_model_den.predict(Xnewden)
    function_costo1 = ufcf/dfcf
    function_costo2 = numerador/denominador
    function_costo2num = function_costo2[0][0]
    function_result = (function_costo2num - function_costo1)/function_costo1
    return function_result

def choice_by_roulette(sorted_population, fitness_sum):
    offset = 0
    normalized_fitness_sum = fitness_sum

    lowest_fitness = apply_function(sorted_population[0])
    if lowest_fitness < 0:
        offset = -lowest_fitness
        normalized_fitness_sum += offset * len(sorted_population)

    draw = random.uniform(0, 1)

    accumulated = 0
    for individual in sorted_population:
        fitness = apply_function(individual) + offset
        probability = fitness / normalized_fitness_sum
        accumulated += probability

        if draw <= accumulated:
            return individual

def sort_population_by_fitness(population):
    return sorted(population, key=apply_function)


def crossover(individual_a, individual_b):
    w1a = individual_a["w1"]
    w2a = individual_a["w2"]
    w3a = individual_a["w3"]
    w4a = individual_a["w4"]
    w5a = individual_a["w5"]
    w6a = individual_a["w6"]
    w7a = individual_a["w7"]
    w8a = individual_a["w8"]
    w9a = individual_a["w9"]
    w10a = individual_a["w10"]
    w11a = individual_a["w11"]
    w12a = individual_a["w12"]
    

    w1b = individual_b["w1"]
    w2b = individual_b["w2"]
    w3b = individual_b["w3"]
    w4b = individual_b["w4"]
    w5b = individual_b["w5"]
    w6b = individual_b["w6"]
    w7b = individual_b["w7"]
    w8b = individual_b["w8"]
    w9b = individual_b["w9"]
    w10b = individual_b["w10"]
    w11b = individual_b["w11"]
    w12b = individual_b["w12"]

    result_crossover = {"w1": (w1a + w1b) / 2, "w2": (w2a + w2b) / 2, "w3": (w3a + w3b) / 2, "w4": (w4a + w4b) / 2, 
    "w5": (w5a + w5b) / 2, "w6": (w6a + w6b) / 2, "w7": (w7a + w7b) / 2, "w8": (w8a + w8b) / 2,
    "w9": (w9a + w9b) / 2, "w10": (w10a + w10b) / 2, "w11": (w11a + w11b) / 2, "w12": (w12a + w12b) / 2}
    return result_crossover


def mutate(individual):
    min_value = -1
    max_value = 1
    next_w1 = individual["w1"] * (1+random.uniform(min_value, max_value)) 
    next_w2 = individual["w2"] * (1+random.uniform(min_value, max_value))
    next_w3 = individual["w3"] * (1+random.uniform(min_value, max_value))
    next_w4 = individual["w4"] * (1+random.uniform(min_value, max_value))
    next_w5 = individual["w5"] * (1+random.uniform(min_value, max_value))
    next_w6 = individual["w6"] * (1+random.uniform(min_value, max_value))
    next_w7 = individual["w7"] * (1+random.uniform(min_value, max_value))
    next_w8 = individual["w8"] * (1+random.uniform(min_value, max_value))
    next_w9 = individual["w9"] * (1+random.uniform(min_value, max_value))
    next_w10 = individual["w10"] * (1+random.uniform(min_value, max_value))
    next_w11 = individual["w11"] * (1+random.uniform(min_value, max_value))
    next_w12 = individual["w12"] * (1+random.uniform(min_value, max_value))

    result_mutation = {"w1": next_w1, "w2": next_w2, "w3": next_w3, "w4": next_w4, "w5": next_w5,
    "w6": next_w6, "w7": next_w7, "w8": next_w8, "w9": next_w9, "w10": next_w10, "w11": next_w11, "w12": next_w12}
    return result_mutation


def make_next_generation(previous_population):
    next_generation = []
    sorted_by_fitness_population = sort_population_by_fitness(previous_population)
    population_size = len(previous_population)
    fitness_sum = sum(apply_function(individual) for individual in previous_population)

    for i in range(population_size):
        first_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)
        second_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)

        individual = crossover(first_choice, second_choice)
        individual = mutate(individual)
        next_generation.append(individual)

    return next_generation

In [4]:
def main():
    generations = 10
    boundaries_x = read_csv()    
    population = generate_population(boundarie=boundaries_x, size=10)

    i = 1
    while True:
        print(f"🧬 GENERATION {i}")

        for individual in population:
             print(individual, apply_function(individual))

        if i == generations:
            break

        i += 1

        population = make_next_generation(population)

    best_individual = sort_population_by_fitness(population)[-1]
    print("\n🔬 FINAL RESULT")
    print(best_individual, (apply_function(best_individual)*100), "%")

In [5]:
if __name__ == "__main__":
    main()

🧬 GENERATION 1
{'w1': 22997.676562353066, 'w2': 2221.1238084654797, 'w3': 12169.00008075803, 'w4': 2418.2069837950935, 'w5': 888.9391608640774, 'w6': 47983.68076994403, 'w7': 19523.337708251318, 'w8': 2753.5606766602486, 'w9': 8296.966012749, 'w10': 1225.371789554865, 'w11': 1247.252487906184, 'w12': 74894.1140223709} 0.9217831338053898
{'w1': 28991.763613168583, 'w2': 2530.111986628338, 'w3': 14656.01269378858, 'w4': 1026.599316153864, 'w5': 1028.634012643763, 'w6': 46176.62767314965, 'w7': 21460.96299888964, 'w8': 3416.512028345129, 'w9': 9867.18377172825, 'w10': 1249.3144154041297, 'w11': 1138.05588022857, 'w12': 82709.07024539665} 0.7270141417019865
{'w1': 20948.754384142623, 'w2': 2881.267447033682, 'w3': 14114.421767670328, 'w4': 976.0779849449939, 'w5': 548.8056373709435, 'w6': 49565.61118325203, 'w7': 21513.172687245606, 'w8': 3643.159546284268, 'w9': 13935.115328656864, 'w10': 1571.2618544818108, 'w11': 1044.5292411114272, 'w12': 37074.91836379769} 0.5871130998931517
{'w1': 28

{'w1': 19009.868229498275, 'w2': 8332.453401528555, 'w3': 10830.107202103829, 'w4': 1900.7100404424043, 'w5': 241.3539542123524, 'w6': 25188.35249880642, 'w7': 19155.419658049068, 'w8': 3379.020457458231, 'w9': 854.0920496517836, 'w10': 2274.891885797175, 'w11': 1437.2229506163305, 'w12': 10723.95381765163} 1.395443995072728
{'w1': 16064.853300194269, 'w2': 869.5288253455973, 'w3': 5950.883792310447, 'w4': 2294.273429140729, 'w5': 639.9783893418801, 'w6': 88727.26835404092, 'w7': 58944.34837697957, 'w8': 3113.954462214948, 'w9': 11221.439550039975, 'w10': 700.0770219294675, 'w11': 675.6999357353499, 'w12': 13276.7610005355} -23462.210489279656
{'w1': 48129.88570004358, 'w2': 269.07446877581066, 'w3': 16316.84481754834, 'w4': 3448.0150309176192, 'w5': 720.674942845454, 'w6': 5317.848106485907, 'w7': 26138.355800875408, 'w8': 6219.157642201273, 'w9': 19756.188244959776, 'w10': 364.919925651075, 'w11': 783.9263701292589, 'w12': 9417.763010180952} -0.2691967603069202
{'w1': 14578.158494860

🧬 GENERATION 6
{'w1': 12014.093044999521, 'w2': 66.98866890288366, 'w3': 174.39168311731768, 'w4': 3436.652872314266, 'w5': 1802.4889468232527, 'w6': 16444.32637207633, 'w7': 659.562996832842, 'w8': 1909.009071203145, 'w9': 4571.5087453681745, 'w10': 5969.094410979314, 'w11': 4064.7853212281657, 'w12': 195896.73945696405} -0.2691967603069202
{'w1': 7798.284301822159, 'w2': 2502.7588553396213, 'w3': 215.41256064447396, 'w4': 1794.1223201107673, 'w5': 4073.2213468887794, 'w6': 3730.1967731344475, 'w7': 23965.734641481984, 'w8': 6044.712543337201, 'w9': 1488.6921476958012, 'w10': 1487.9115641730234, 'w11': 295.94094965140994, 'w12': 18220.31381026628} -0.2691967603069202
{'w1': 10468.839218566725, 'w2': 10648.015057631526, 'w3': 583.5620809300854, 'w4': 1061.2818877874997, 'w5': 3021.378169787644, 'w6': 25634.89186068229, 'w7': 11340.9421316204, 'w8': 10955.851776221474, 'w9': 2922.6798399021163, 'w10': 123.23074562033477, 'w11': 709.2109152458484, 'w12': 22564.94566049345} -1.00000739605

{'w1': 16470.120357929245, 'w2': 11833.568389456166, 'w3': 320.5492461438121, 'w4': 1379.49015897967, 'w5': 148.25515575697352, 'w6': 37311.774019936354, 'w7': 6704.291845924406, 'w8': 3138.858900419775, 'w9': 2099.1308894945787, 'w10': 242.35481338831264, 'w11': 114.68476986464634, 'w12': 29788.47361870497} -62361.71934867996
{'w1': 6341.295976212629, 'w2': 1170.5836576577997, 'w3': 200.97537338616638, 'w4': 824.3271448371843, 'w5': 795.1108938825149, 'w6': 39952.14139349178, 'w7': 2070.0479631532085, 'w8': 4117.740799881623, 'w9': 1798.257958686284, 'w10': 2718.127213491038, 'w11': 534.8584120887874, 'w12': 12152.263876326122} -1.0000428489010327
{'w1': 3196.1349931274117, 'w2': 10786.99721528253, 'w3': 1279.424209862144, 'w4': 1370.3253838688063, 'w5': 21.85452750685712, 'w6': 5779.544789144497, 'w7': 132.23042204779438, 'w8': 8062.307407811239, 'w9': 10058.115518186238, 'w10': 370.7625261309815, 'w11': 1124.3739517083895, 'w12': 22722.489357164683} -0.2691967603069202
{'w1': 9361.4


🔬 FINAL RESULT
{'w1': 2294.718487823313, 'w2': 3952.0446046868037, 'w3': 604.1277117700346, 'w4': 1725.9161166499166, 'w5': 269.87849922056864, 'w6': 17341.650080055348, 'w7': 2069.339435559571, 'w8': 737.0464561891231, 'w9': 843.8953925184228, 'w10': 910.9931048307515, 'w11': 1065.2357079990932, 'w12': 31241.06640930358} -26.91967603069202 %
